In [4]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os

def Plot(data,signal,background,histograms,final_state="Z#rightarrow #mu#mu"):
    samples = data.copy()
    samples.update(background)
    samples.update(signal)

    for i in histograms:
        print(i)
        for s in samples:
            file = ROOT.TFile.Open(samples[s][0],"READ")
            hist = file.Get(i)
            hist.SetDirectory(0)
            samples[s].append
            samples[s][2]=hist # add histogram (TH1F) to list of samples
            file.Close()

        ###### SETTING THE COLOURS ######

        for s in samples:
            if s=="Data":
                samples[s][2].SetMarkerStyle(20)
                samples[s][2].SetMarkerSize(0.5)
                samples[s][2].SetLineColor(samples[s][1])
            else :
                samples[s][2].SetFillColor(samples[s][1])
                samples[s][2].SetLineColor(samples[s][1]+1)

        ####################### CREATING MC AND STACK HISTOGRAM ########################        
        
        hs = ROOT.THStack("hs","")
        mc = samples["Signal"][2].Clone()
        for s in samples:
            if s!="Data":
                hs.Add(samples[s][2])
                if s!="Signal":
                    mc.Add(samples[s][2],1)

        ############### DEFINING RATIOS ###############

        ratio = mc.Clone()
        ratio.Divide(samples["Data"][2])


        ##### DRAWING TOP PAD, SETTING MARGINS #######
        
        gStyle.SetOptStat(1111111)
        gStyle.SetStatY(0.97);                
        gStyle.SetStatX(1.0);
        gStyle.SetStatW(0.12);                
        gStyle.SetStatH(0.12);

        canvas = ROOT.TCanvas("canvas2")
        canvas.cd()

        pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
        pad1.SetTopMargin(0.03)
        pad1.SetRightMargin(0.03)
        pad1.SetLeftMargin(0.08)
        pad1.SetBottomMargin(0.0)
        pad1.Draw ()
        pad1.cd ()

        ###### SETTING STATS BOX POSITION ######
        
        gStyle.SetStatY(0.93);                
        gStyle.SetStatX(0.99);
        gStyle.SetStatW(0.1);                
        gStyle.SetStatH(0.1);

        
        samples["Data"][2].Draw("pe same")
        hs.Draw("HIST same")
        samples["Data"][2].Draw("pe same")
        samples["Data"][2].Draw("sameaxis")

        #### Set the Y Axis to logarithmic if desired
        #pad1.SetLogy() 
        
        s=samples["Data"][2].GetXaxis().GetBinLowEdge(1)
        e=samples["Data"][2].GetXaxis().GetBinUpEdge(samples["Data"][2].GetNbinsX())
        
        samples["Data"][2].GetYaxis().SetRangeUser(0.1 ,1.5*samples["Data"][2].GetBinContent(samples["Data"][2].GetMaximumBin()))
        samples["Data"][2].GetXaxis().SetRangeUser(s,e)
        hs.GetXaxis().SetRangeUser(s,e)
        
        samples["Data"][2].GetYaxis().SetTitle("N Events")

        legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
        for s in samples:
            legend.AddEntry(samples[s][2],s)
        legend.SetNColumns(3)
        gStyle.SetLegendBorderSize(0)
        legend . SetLineWidth (0)
        legend . Draw ()

        samples["Data"][2].SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.DrawLatex(0.9,0.7,final_state)
        
        s=samples["Data"][2].GetXaxis().GetBinLowEdge(1)
        e=samples["Data"][2].GetXaxis().GetBinUpEdge(samples["Data"][2].GetNbinsX())
        
        max_ratio = ratio.GetMaximum()
        if max_ratio > 3:
            max_ratio = 3
        if max_ratio < 1.5:
            max_ratio = 1.5

        min_ratio = ratio.GetMinimum()
        if min_ratio > 0.7:
            min_ratio = 0.5
            
        canvas.cd()
        pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.35)
        pad2.SetRightMargin(0.03)
        pad2.SetLeftMargin(0.08)
        pad2.SetTopMargin(0)
        pad2.SetBottomMargin(0.0)
        pad2.Draw ()
        pad2.cd ()
        ratio.SetFillColorAlpha(ROOT.kBlue,0.35)
        ratio.Draw ("E2")
        ratio.SetTitle("")
        ratio.SetStats(0)
        ratio . GetYaxis (). SetRangeUser (min_ratio ,max_ratio)
        ratio . GetXaxis (). SetRangeUser (s ,e)
        ratio . GetYaxis (). SetTitle ("MC/DATA")
        ratio . GetYaxis (). SetTitleSize (0.15)
        ratio . GetYaxis (). SetTitleOffset (0.25)
        ratio . GetXaxis (). SetTitleSize (0.09)
        ratio.GetXaxis().SetLabelSize(0.10)
        ratio.GetYaxis().SetLabelSize(0.08)
        ratio.SetMarkerStyle(8)
        ratio.SetMarkerSize(0.6)


        ###### SETTING ALL THE HORIZONTAL DASHED LINES #######

        line = ROOT . TLine (s ,1 ,e,1)
        line . SetLineColor ( ROOT . kBlack )
        line . SetLineWidth (2)

        separators = []
        resolution = 10 # In percentage
        j=0
        range_sep = []
        while j < (int(max_ratio)+1):
            step = resolution/100
            if j > min_ratio:
                range_sep.append(j)
            j = j + step

        for k in range (len(range_sep)):
            sep = ROOT.TLine(s ,range_sep[k],e,range_sep[k])
            sep.SetLineColor(ROOT.kBlack)
            sep.SetLineWidth(1)
            sep.SetLineStyle(2)
            separators.append(sep)

        line . Draw (" same ")   
        for separ in separators:
            separ.Draw("same")
        
        file_name = i+".pdf"

        canvas.Print(file_name)

In [15]:
#### HISTOGRAMS FOR PLOTTING AND SIGNAL SAMPLE NAME
histograms={
"tau_pt",
"delta_phi",
"reco_mass",
}
signal_sample = 'Ztautau_Sherpa.root'

In [16]:
Data = {"Data":["Data.root",ROOT.kBlack,0]}   
Signal = {"Signal":[signal_sample,ROOT.kOrange+7,0]} 
Background = {"Higgs":["Higgs.root",ROOT.kRed,0],"Zjets":["Zjet.root",ROOT.kViolet,0],
"Wjets":["Wjets.root",ROOT.kGreen,0],"ttbar":["ttbar.root",ROOT.kYellow,0],
"VV":["VV.root",ROOT.kBlue,0],"SingleTop":["singletop.root",ROOT.kCyan,0]}

Plot(Data,Signal,Background,histograms,final_state="Z#rightarrow #tau#tau")

tau_pt
reco_mass
delta_phi


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file tau_pt.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file reco_mass.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file delta_phi.pdf has been created
